<a href="https://colab.research.google.com/github/LukrecijaTudor/NLP/blob/main/B_E_R_T.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pycld2

     |████████████████████████████████| 41.4MB 75kB/s 
  Created wheel for pycld2: filename=pycld2-0.41-cp36-cp36m-linux_x86_64.whl size=9833565 sha256=bb2b7622398b23f6ad03c3d64222be08cd44ae9a217e8d8d1090206f985d17b7
  Stored in directory: /root/.cache/pip/wheels/c6/8f/e9/08a1a8932a490175bd140206cd86a3dbcfc70498100de11079
Successfully built pycld2


In [ ]:
pip install transformers

     |████████████████████████████████| 1.8MB 17.0MB/s 
     |████████████████████████████████| 890kB 50.8MB/s 
     |████████████████████████████████| 2.9MB 51.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=d13e3d0dd8dc8a8fb9b3cbfb7d50f9ac4694f7b3afc38059043b33149e52ea1d
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import pandas as pd
import pycld2 as cld2
#import ndjson
from sklearn.model_selection import train_test_split

In [ ]:
acronyms = pd.read_excel("/content/kratice.xlsx")
acronyms = acronyms.rename(columns={'kratice':'acronym','Unnamed: 1':'replacement'})
document = pd.read_csv("/content/qa06_all.csv")
document = document.rename(columns={'qa06id': 'id', 'qa06name':'title', 'qa06wher':'location','qa06dsc':'report'})
document_risk=pd.read_csv("/content/qa06_only_having_risk_valuesl.csv")
document_risk = document_risk.rename(columns={'qa06id': 'id', 'qa06name':'title', 'qa06wher':'location','qa06dsc':'report','ty26colo':'label','ty26fakt':'factor'})
def acronyms_to_words (txt):
    #acronyms = pd.read_excel("/Users/Lukre/Desktop/D/kratice.xlsx")
    #acronyms = acronyms.rename(columns={'kratice':'acronym','Unnamed: 1':'replacement'})
    for i in range (len(acronyms)):
        txt=txt.replace(' ' + acronyms.acronym[i] + ' ',' ' + acronyms.replacement[i] + ' ')
    return txt

def isNaN(string):
    return string != string
def prep_doc__for_bert_classification(document):
    
  doc = pd.DataFrame(columns=['report','accident','label'])
    
  for i in range(len(document)):
       
      if isNaN(document.report[i]):
          continue
      isReliable, textBytesFound, details=cld2.detect(document.report[i])
      
      if not isReliable:
                  continue
      if details[0][0]!='ENGLISH':
          continue
      txt=acronyms_to_words (document.report[i])
      if (document.factor[i]==1):
                pom_2=0
                pom='no accident outcome'
      
      if (document.factor[i] in (2,4,20,100)):
                pom_2=1
                pom='minor injuries or damage'
      if (document.factor[i] in (10,50,21,102,101,502,500)):
                pom_2=2
                pom='major or catastrophic accident'

      doc=doc.append({'report': txt,'accident': pom,'label': pom_2},ignore_index=True)

  return doc
def train_val_test_split_ (doc,label,rnd=42,size=0.2):
  X_train_1, X_test, y_train_1, y_test = train_test_split(doc.index.values, 
                                                  doc.label.values, 
                                                  test_size=size, 
                                                  random_state=rnd, 
                                                  stratify=doc.label.values)
  
  pom = pd.DataFrame(columns=['X','y'])
  pom['X']=X_train_1
  pom['y']=y_train_1

  X_train, X_val, y_train, y_val = train_test_split(pom.X.values, 
                                                  pom.y.values, 
                                                  test_size=size, 
                                                  random_state=rnd, 
                                                  stratify=pom.y.values) 
  doc['data_type'] = ['not_set']*doc.shape[0]


  doc.loc[X_train, 'data_type'] = 'train'
  doc.loc[X_val, 'data_type'] = 'val' 
  doc.loc[X_test, 'data_type'] = 'test'
  return doc

In [ ]:
doc_bert = prep_doc__for_bert_classification (document_risk)

In [ ]:
texts=doc_bert['report'].to_list()
labels=doc_bert['accident'].to_list()

In [ ]:
main_txt, test_txt, main_labels, test_labels = train_test_split(texts, labels, test_size=0.1, random_state=1)
train_txt, val_txt, train_labels, val_labels = train_test_split(main_texts, rest_labels, test_size=0.1, random_state=1)

print("Train size:", len(train_texts))
print("Val size:", len(dev_texts))
print("Test size:", len(test_texts))

Train size: 2308
Dev size: 257
Test size: 286


In [ ]:
class_names = list(set(labels))
label_ids = {label: id for id, label in enumerate(class_names)}
print(label_ids)

{'minor injuries or damage': 0, 'major or catastrophic accident': 1, 'no accident outcome': 2}


In [ ]:
from transformers import BertTokenizer

BERT_MODEL = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL)

In [ ]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(BERT_MODEL, num_labels = len(label_ids))
model.to(device)

In [ ]:
import logging
import numpy as np

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger(__name__)

#MAX_SEQ_LENGTH=100
MAX_SEQ_LENGTH=256

class BertInput(object):

    def __init__(self, text, input_ids, input_mask, segment_ids, label_id):
        self.text = text
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id
        

def input_data(example_texts, example_labels, label_ids, max_seq_length, tokenizer, verbose=0):
    
    input_items = []
    examples = zip(example_texts, example_labels)
    for (ex_index, (text, label)) in enumerate(examples):

        input_ids = tokenizer.encode(f"[CLS] {text} [SEP]")
        if len(input_ids) > max_seq_length:
            input_ids = input_ids[:max_seq_length]

        segment_ids = [0] * len(input_ids)

        input_mask = [1] * len(input_ids)

        padding = [0] * (max_seq_length - len(input_ids))
        input_ids += padding
        input_mask += padding
        segment_ids += padding

        #provjera valjanosti
        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length

        label_id = label_ids[label]

        input_items.append(
            BertInput(text=text,
                          input_ids=input_ids,
                          input_mask=input_mask,
                          segment_ids=segment_ids,
                          label_id=label_id))

        
    return input_items

train_features = input_data(train_txt, train_labels, label_ids, MAX_SEQ_LENGTH, tokenizer, verbose=0)
dev_features = input_data(val_texts, val_labels, label_ids, MAX_SEQ_LENGTH, tokenizer)
test_features = input_data(test_txt, test_labels, label_ids, MAX_SEQ_LENGTH, tokenizer)

Token indices sequence length is longer than the specified maximum sequence length for this model (513 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
import pickle 

In [ ]:
  with open ('train_input_bert.csv','wb') as f:
    pickle.dump(train_features,f)
  with open ('val_input_bert.csv','wb') as g:
    pickle.dump(dev_features,g)
  with open ('test_input_bert.csv','wb') as h:
    pickle.dump(test_features,h)
  with open ('train_input_bert.csv','rb') as i:
    d=pickle.load(i)
  with open ('val_input_bert.csv','rb') as j:
    e=pickle.load(j)
  with open ('test_input_bert.csv','rb') as k:
    l=pickle.load(k)


In [ ]:
import json
with open('train_input_bert.txt', 'w') as f:
    f.write(json.dumps(train_features))
with open('val_input_bert.txt', 'w') as f:
    f.write(json.dumps(dev_features))
with open('test_input_bert.txt', 'w') as f:
    f.write(json.dumps(test_features))

#Now read the file back into a Python list object
#with open('test.txt', 'r') as f:
#    a = json.loads(f.read())

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, SequentialSampler

def get_data_loader(features, max_seq_length, batch_size, shuffle=True): 

    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
    all_label_ids = torch.tensor([f.label_id for f in features], dtype=torch.long)
    data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)

    dataloader = DataLoader(data, shuffle=shuffle, batch_size=batch_size)
    return dataloader

BATCH_SIZE = 16

train_dataloader = get_data_loader(train_features, MAX_SEQ_LENGTH, BATCH_SIZE, shuffle=True)
val_dataloader = get_data_loader(val_features, MAX_SEQ_LENGTH, BATCH_SIZE, shuffle=False)
test_dataloader = get_data_loader(test_features, MAX_SEQ_LENGTH, BATCH_SIZE, shuffle=False)

In [ ]:
def evaluation(model, dataloader):
    model.eval()
    
    eval_loss = 0
    nb_eval_steps = 0
    predicted_labels, correct_labels = [], []

    for step, batch in enumerate(tqdm(dataloader, desc="Evaluation iteration")):
        batch = tuple(t.to(device) for t in batch)
        input_ids, input_mask, segment_ids, label_ids = batch

        with torch.no_grad():
            tmp_eval_loss, logits = model(input_ids, attention_mask=input_mask,
                                          token_type_ids=segment_ids, labels=label_ids,return_dict=False)

        outputs = np.argmax(logits.to('cpu'), axis=1)
        label_ids = label_ids.to('cpu').numpy()
        
        predicted_labels += list(outputs)
        correct_labels += list(label_ids)
        
        eval_loss += tmp_eval_loss.mean().item()
        nb_eval_steps += 1

    eval_loss = eval_loss / nb_eval_steps
    
    correct_labels = np.array(correct_labels)
    predicted_labels = np.array(predicted_labels)
        
    return eval_loss, correct_labels, predicted_labels

In [ ]:
from transformers.optimization import AdamW
from transformers.optimization import get_linear_schedule_with_warmup

GRADIENT_ACCUMULATION_STEPS = 1
NUM_TRAIN_EPOCHS =6 #20
LEARNING_RATE = 1e-5
WARMUP_PROPORTION = 0.1
MAX_GRAD_NORM = 5

num_train_steps = int(len(train_dataloader.dataset) / BATCH_SIZE / GRADIENT_ACCUMULATION_STEPS * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(WARMUP_PROPORTION * num_train_steps)

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

optimizer = AdamW(optimizer_grouped_parameters, lr=LEARNING_RATE, correct_bias=False)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_train_steps)

In [ ]:
import torch
import os
from tqdm import trange
from tqdm import tqdm_notebook as tqdm
from sklearn.metrics import classification_report, precision_recall_fscore_support

loss_history = []
no_improvement = 0
for _ in trange(int(NUM_TRAIN_EPOCHS), desc="Epoch"):
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(tqdm(train_dataloader, desc="Training iteration")):
        batch = tuple(t.to(device) for t in batch)
        input_ids, input_mask, segment_ids, label_ids = batch

        outputs = model(input_ids, attention_mask=input_mask, token_type_ids=segment_ids, labels=label_ids)
        loss = outputs[0]

        if GRADIENT_ACCUMULATION_STEPS > 1:
            loss = loss / GRADIENT_ACCUMULATION_STEPS

        loss.backward()
        tr_loss += loss.item()

        if (step + 1) % GRADIENT_ACCUMULATION_STEPS == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), MAX_GRAD_NORM)  
            
            optimizer.step()
            optimizer.zero_grad()
            scheduler.step()
            
    val_loss, _, _ = evaluation(model, val_dataloader)
    
    print("Loss history:", loss_history)
    print("Val loss:", val_loss)
    
    if len(loss_history) == 0 or val_loss < min(loss_history):
        no_improvement = 0
        model_to_save = model.module if hasattr(model, 'module') else model
    else:
        no_improvement += 1
     
    loss_history.append(val_loss)

In [ ]:
model = model_to_save
model.to(device)

model.eval()

_, train_correct, train_predicted = evaluation(model, train_dataloader)
_, val_correct, val_predicted = evaluation(model, val_dataloader)
_, test_correct, test_predicted = evaluation(model, test_dataloader)

print("Training performance:", precision_recall_fscore_support(train_correct, train_predicted, average="micro"))
print("Development performance:", precision_recall_fscore_support(val_correct, val_predicted, average="micro"))
print("Test performance:", precision_recall_fscore_support(test_correct, test_predicted, average="micro"))

bert_accuracy = np.mean(test_predicted == test_correct)

print(classification_report(test_correct, test_predicted, target_names=target_names))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  



Training performance: (0.8994800693240901, 0.8994800693240901, 0.8994800693240901, None)
Development performance: (0.5797665369649806, 0.5797665369649806, 0.5797665369649806, None)
Test performance: (0.6538461538461539, 0.6538461538461539, 0.6538461538461539, None)
                                precision    recall  f1-score   support

major or catastrophic accident       0.50      0.12      0.20        24
      minor injuries or damage       0.72      0.76      0.74       177
           no accident outcome       0.53      0.58      0.55        85

                      accuracy                           0.65       286
                     macro avg       0.58      0.49      0.50       286
                  weighted avg       0.64      0.65      0.64       286



In [ ]:
import sklearn
sklearn.metrics.confusion_matrix(test_correct, test_predicted, labels=None, sample_weight=None, normalize=None)

array([[  3,  17,   4],
       [  3, 135,  39],
       [  0,  36,  49]])

In [ ]:
y_test = test_correct
y_pred = test_predicted

In [ ]:
from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(y_test, y_pred)
print('Confusion Matrix\n')
print(confusion)

#importing accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('\nAccuracy: {:.2f}\n'.format(accuracy_score(y_test, y_pred)))

print('Micro Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='micro')))
print('Micro Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='micro')))
print('Micro F1-score: {:.2f}\n'.format(f1_score(y_test, y_pred, average='micro')))

print('Macro Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='macro')))
print('Macro Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='macro')))
print('Macro F1-score: {:.2f}\n'.format(f1_score(y_test, y_pred, average='macro')))

print('Weighted Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_test, y_pred, average='weighted')))

Confusion Matrix

[[  3  17   4]
 [  3 135  39]
 [  0  36  49]]

Accuracy: 0.65

Micro Precision: 0.65
Micro Recall: 0.65
Micro F1-score: 0.65

Macro Precision: 0.58
Macro Recall: 0.49
Macro F1-score: 0.50

Weighted Precision: 0.64
Weighted Recall: 0.65
Weighted F1-score: 0.64
